# DVE Rental SQL Problems

## Setup

In [5]:
# import psycopg2
import pandas as pd
import numpy as np
# import pymysql
import sys

In [ ]:
try:
    conn = psycopg2.connect("dbname='dvdrental' user='' host='localhost' password=''")
except:
    print("I am unable to connect to the database")

In [ ]:
category = pd.read_sql_query('SELECT * from category',con=conn)
film_category = pd.read_sql_query('SELECT * from film_category',con=conn)
film = pd.read_sql_query('SELECT * from film',con=conn)
language = pd.read_sql_query('SELECT * from language',con=conn)
film_actor = pd.read_sql_query('SELECT * from film_actor',con=conn)
inventory = pd.read_sql_query('SELECT * from inventory',con=conn)
rental = pd.read_sql_query('SELECT * from rental',con=conn)
payment = pd.read_sql_query('SELECT * from payment',con=conn)
staff = pd.read_sql_query('SELECT * from staff',con=conn)
actor = pd.read_sql_query('SELECT * from actor',con=conn)
customer = pd.read_sql_query('SELECT * from customer',con=conn)
address = pd.read_sql_query('SELECT * from address',con=conn)
city = pd.read_sql_query('SELECT * from city',con=conn)
country = pd.read_sql_query('SELECT * from country',con=conn)
store = pd.read_sql_query('SELECT * from store',con=conn)

![](_pic/DVD-Rental-ER-Diagram.png)

# Question, Idea & Solution

---

> ### Get a list of actors with the first name Chris, Cameron, or Cuba.


**> idea:**
Use the ```SELECT``` & ```WHERE``` command to filter out the ideal rows.
 
**> solution:**

```
    SELECT * from actor where first_name in ('Chris', 'Cameron', 'Cuba'); 
```    

**> note:**

* `IN` lets you specify a lot of values that you would otherwise join together with an `OR` statement
* We can test for `NULL` with `IS NULL`. If we want to filter out '<NA>' rows, we can use `IS NOT NULL` in `WHERE`.
* `WHERE` operators include:

| Operator | Description |
|:---:|:---|
| = | Equal |
| > | Greater than |
| < | Less than |
| >= | Greater than or equal |
| <= | Less than or equal |
| <> or != | Not equal |
| AND | Logical operator AND |
| OR | Logical operator OR |

In [ ]:
actor[actor['first_name'].isin(['Chris','Cameron', 'Cuba'])]

In [ ]:
actor.query('first_name=="Chris" | first_name=="Cameron" | first_name=="Cuba" ')

In [ ]:
actor[(actor['first_name'] == "Chris") | (actor['first_name'] == "Cameron")]

**pandas note**

* we can filter the dataframe using `.query('colname2>10 & colname2=="abc"')`. For or, using `|` 

---

> ### What are the different rental durations that the store allows?

**> idea:**
Use `DISTINCT` operator together in `SELECT`

**> solution:**
```
select distinct rental_duration from film;
```




In [ ]:
film.rental_duration.unique()

---
> ### How many films are rated NC-17? How many are rated PG or PG-13?

**> idea:**
filter rows from films that are rated "NC-17", and use count command to see how many rows there are.
If we want to check "How many are rated PG or PG-13?", in the where command, we can use `in` to filter rows that are in `('PG', 'PG-13')`

**> solution:**
```
select count(*) from film where rating in ('PG','PG-13');
```

In [ ]:
film[film['rating'].isin(['PG','PG-13'])].groupby(['rating']).agg({'film_id':'count'}).reset_index().rename(columns={'film_id': 'count'})

**pandas note**

* To summarise after groupby, we can use `.agg({'colname':'count'})`
* To rename colnames, we can use `.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})`. Remember to use `columns` with s.

Here are a list of method that we can use for `agg`

| method |  |
|:---:|:---|
| 'sum' or np.sum | return the sum |
| np.mean | Return the mean |
| 'count' | return the count |
| 'nunique' | Return the count of a number of different values |
| 'max' | Return the maximum value |
| 'min' | Return the minimum value |
| np.std | Return the standard deviation |
| np.var | Return the variance |


---

> ### How many different customers have entries in the rental table?

**> idea:**
Use distinct to get the unique id from the field `customer_id`, and then combine with `count` command to get the count number.


**> solution:**
```
select count(distinct customer_id) from rental;
```

In [ ]:
rental['customer_id'].nunique()

---

> ### What are the IDs of the last 5 customers to return a rental?

**> idea:**
order the `rental` table by `return_date` in descending order and get the `customer_id` of the first 3 rows


**> solution:**

```
select customer_id, return_date from rental where customer_id=251 order by return_date DESC;
```

```
select customer_id, count(*), max(return_date) from rental group by customer_id;
```


```
select customer_id, return_date from rental where return_date is not null order by return_date DESC limit 5;
```

**> better solution:**
Since a customer_id can show up twice in the first five rows. To deduplicated this, we can do the following. Even though the result is the same, but it's a better solution.
```
select customer_id, recent_date from (
  select customer_id, max(return_date) as recent_date from rental where return_date is not null group by customer_id 
) as tbl1 order by recent_date DESC limit 5;
```



**> note:**

* We can add `DESC` after `order by` to get the rows in descending order. If not specify, it will be ascending order.
* If we want to filter out '<NA>' rows, we can use `IS NOT NULL` in `WHERE`.

In [ ]:
rental.groupby(['customer_id']).agg({'return_date':'max'}).reset_index().sort_values(by=['return_date'], ascending=False).head(5)

**pandas note**

* to sort values, we can use `.sort_values(by=['colname1','colnames2'], ascending=False)` to return value by descending order. 

---

> ### Does the average replacement cost of a film differ by rating?

**> idea:**
We will want to `group by` rating and calculate the average replacement cost.

**> solution:**
```
select rating, avg(replacement_cost) from film group by rating;
```


**> note:**

* All columns except the columns applied with some calculation in the `SELECT` part of the statement have to be in the GROUP BY part, or you'll get an error.
* The [aggregate function](https://www.postgresql.org/docs/9.5/static/functions-aggregate.html) that can be used with `GROUP BY` include:

| Name | Description |
|:---:|:---|
| avg() | Return the average value of the argument |
| count() | Return a count of the number of rows returned |
| count(distinct) | Return the count of a number of different values |
| max() | Return the maximum value |
| min() | Return the minimum value |
| sum() | Return the sum |
| stddev_pop | Return the population standard deviation |
| stddev(), stddev_samp() | Return the sample standard deviation |
| var_pop() | Return the population standard variance |
| variance(), var_samp() | Return the sample variance |
| array_agg() | input arrays concatenated into array of one higher dimension  |
| json_agg | aggregates values as a JSON array | 

In [ ]:
film.groupby(['rating']).agg({'replacement_cost':np.mean}).reset_index()

---

> ### Select film title that have "Dragon" in them.

**> idea:**
Using some command with `where` to filter our title with Dragon in them. It turns out the command we need is `similar to` combining with some wildcards.


**> solution:**
```
select * from film where title similar to '[C|F]%Dragon%';
```


**> note:**

* `%` : The percent sign represents zero, one, or multiple characters
* `_` : The underscore represents a single character
* `'[bsp]%'`: The following SQL statement selects the target value starting with "b", "s", or "p"
* `'[!bsp]%'`: The following SQL statement selects the target value NOT starting with "b", "s", or "p"
    * We can also use `NOT LIKE '[bsp]%'`
* `'[a-c]%'`: The following SQL statement selects the target value starting with "a", "b", or "c"
* `'[a|c]%'`: denotes alternation (either of two alternatives). match value starting with either "a" or "c".
* For more matching command, check [[here]](https://www.postgresql.org/docs/9.0/static/functions-matching.html).

In [ ]:
film[film['title'].str.contains("Dragon")]

**pandas note**

* we can use `.str.contains("pattern")` within filter to filter by string pattern.

---

> ### Which store (store_id) has the most customers whose first name starts with M?

**> idea:**
We will probably need to filter out customer with 'first_name' starting with M, then group by `store_id`, and then count the distinct `customer_id`. (Actually, since `customer_id` is the primary key of the table, we don't need `distinct`.)


**> solution:**
```
select store_id, count(distinct customer_id) from customer where first_name similar to 'M%' group by store_id order by count(*) DESC;
```

In [ ]:
customer[customer['first_name'].str.startswith("M")].groupby('store_id').agg({'customer_id':'count'}).reset_index().rename(columns={'customer_id':'count'})

---

> ### Are there any customers with the same last name?

**> idea:**
We will probably want to `group by` `last_name` to check what are the `last_name` that is used ore than once. To do this, we will need to combine with `Having`

**> solution:**
```
select last_name, count(*) from customer group by last_name having count(*)>1;
```
It turns out that there is no customers with the same last name.

In [ ]:
customer.groupby(['last_name'])['customer_id'].count().reset_index().rename(columns={'customer_id':'count'}).query("count>1")

**pandas note**

* we can filter the dataframe using `.query('colname>10')`

---

> ### Which film (id) has the most actors?

**> idea:**
To do this, we will need to use `film_actor` table, `group by` `film_id` and count `actor_id`. Also, we'll want to sort by the count value.

**> solution:**
```
select film_id, count(actor_id) from film_actor group by film_id order by count(actor_id) DESC;
```

In [ ]:
film_actor.groupby(['film_id'])['actor_id'].count().reset_index().rename(columns={'actor_id':'count'}).sort_values(by=['count'], ascending = False).head()

---

> ### What films are actors with ids 129 and 195 in together?

**> idea:**
What we will need to do is first select films that `actor_id 129`is in. Then, out of those films, we then filter out films that `actor_id 195` is also in. Because we need to do twice, we'll need to use subquery

**> solution:**

```
select film_id from film_actor where film_id IN (select film_id from film_actor where actor_id=129) and actor_id=195;
```

In [ ]:
a=film_actor.query('actor_id==129')
b=film_actor.query('actor_id==195')
c=pd.merge(a,b,on='film_id',how='left')

c[(c.actor_id_y).notnull()]

**pandas note**

* to drop rows of Pandas DataFrame whose value in certain columns is NaN. We can use `df[df.colname.notnull()]` or `df[~df.colname.isnull()]`
* to merge dataframes, we can use `pd.merge(df1,df2,  on='ad_id', how='left')`. 
    * how : {'left', 'right', 'outer', 'inner'}, default 'inner'

---

> ### How many actors are in more films than actor id 47? 

**> idea:**
We'll firstly need to know how many film do actor 47 perform in. We then need to know how many films do each actors perform in. Then, we filter out the actors that performs more films than actor 47.

Having the above table, we then count how many rows in the table.

**> solution:**
```
select count(*) from 
    (Select actor_id, count(*) from film_actor group by actor_id 
      having 
          count(*) > (select count(*) from film_actor where actor_id=47)
    ) tbl1;
```

**> note:**

* If we want to select from some temporary generated result set, we'll need to give our subquery a name.

In [ ]:
id47 = film_actor.query('actor_id==47')['film_id'].count()

film_actor.groupby('actor_id').agg({'film_id':'nunique'}).reset_index().rename(columns={'film_id':'count'}).query('count>@id47').head()


**pandas note**

* to pass a variable into a query, we can use `.query('colname>@variable')`

---

> ### Join the customer and payment tables together with an inner join; select customer id, name, amount, and date and order by customer id. Then join the staff table to them as well to add the staff's name.

**> idea:**
This is just to practice some command for inner join

**> solution:**
```
select * from 
  staff as s, 
  (select c.customer_id, c.first_name, c.last_name, p.amount, c.create_date, p.staff_id 
  from customer as c, payment as p where c.customer_id=p.customer_id 
  order by customer_id) tbl1 
where s.staff_id=tbl1.staff_id; 
```
```
SELECT
 customer.customer_id,
 customer.first_name customer_first_name,
 customer.last_name customer_last_name,
 staff.first_name staff_first_name,
 staff.last_name staff_last_name,
 amount,
 payment_date
FROM
 customer
INNER JOIN payment ON payment.customer_id = customer.customer_id
INNER JOIN staff ON payment.staff_id = staff.staff_id
ORDER BY
 customer.customer_id;
```

**> note:**

* We can use `WHERE r1.id=r2.id` or `INNER JOIN r2 on r1.id=r2.id` to do a inner join
* When using `INNER JOIN` and `WHERE` together to filter something, `WHERE` should be used after the `INNER JOIN` command.

In [ ]:
pd.merge(customer,payment, on='customer_id',how='inner').rename(columns={'first_name':'customer_first','last_name':'customer_last'}).merge(staff,on='staff_id')[['customer_id','customer_first','customer_last','amount','create_date','staff_id','first_name','last_name']].head(5)


---

> ### Create a list of addresses that includes the name of the city instead of an ID number and the name of the country as well.

**> idea:**
We'll want to do a inner join using address and city, also address and country

**> solution:**
```
select 
  address, 
  address2,
  city,
  country
from 
  address as a
inner join city on a.city_id=city.city_id
inner join country on city.country_id=country.country_id;
```

In [ ]:
address.merge(city,on='city_id',how='left').merge(country,on='country_id',how='left')[['address','city','country']].head(5)


---

> ### Which two actors have been in the most films together?

**> idea:**
It is a harder problem. To start, we will probably need to get a dataframe with actor_id 1 and actor_id 2 and the film_id. With this dataframe, we can then count how many rows do each of the two actor_id appears together in the dataframe. We can get the count by using `group by`.

**> solution:**
```
SELECT a.actor_id, b.actor_id, count(*)
FROM film_actor a, film_actor b -- join the table to itself
WHERE a.film_id=b.film_id -- on the film id
      AND a.actor_id > b.actor_id -- avoid duplicates and matching to the same actor
GROUP BY a.actor_id, b.actor_id
ORDER BY count(*) DESC;
```

In [ ]:
film_actor.head()

In [ ]:
film_actor.merge(film_actor,on='film_id').query('actor_id_x<actor_id_y').groupby(['actor_id_x','actor_id_y']).agg({'film_id':'count'}).reset_index().sort_values(by='film_id',ascending=False).head(5)


---

> ### Get a list of the names of customers who have spent more than $150, along with their total spending.

**> idea:**
We can firstly get the `customer_id` that has spent more than $150 using payment. Then we join the result with the `customer` t o get the name.

We can also firstly join `customer` to get all the names and then `group by` to filter out all the customers with payment over $150. The first version is more efficient, since it filter out before doing the join.

**> solution:**

```
select c.first_name, c.last_name, c.customer_id, tbl1.sum
from customer as c,
  (select 
    p.customer_id, sum(amount) 
  from payment as p 
  group by customer_id having sum(amount) > 150) tbl1
where c.customer_id = tbl1.customer_id
order by sum DESC;
```

```
select 
  c.first_name,
  c.last_name,
  sum(amount)
from 
  customer as c,
  payment as p
where 
  c.customer_id=p.customer_id 
group by  
  c.first_name, c.last_name
having 
  sum(amount) > 150
order by 
  sum(amount) DESC;
```

In [ ]:
payment.groupby(['customer_id']).agg({'amount':'sum'}).reset_index().query('amount>150').merge(customer, on='customer_id', how='inner')[['customer_id','first_name','last_name','amount']].sort_values(by='amount',ascending=False)


In [ ]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='world')

In [ ]:
city=pd.read_sql_query('SELECT * from city',con=connection)
country=pd.read_sql_query('SELECT * from country',con=connection)
language=pd.read_sql_query('SELECT * from country',con=connection)

In [ ]:
city.head()

In [ ]:
country.head()

In [ ]:
language.head()

> ### Find the largest country in terms of population in each continent

**> idea:**
1. First have a table with column `continent, country, population`
2. Get the country with the largest population for each continent from the table.

The second part is tricky. We can not just `group by` continent and then select the max country by population. It's not that straight forward. 

We probably need to join the table by itself and then use some condition to get the conutry with the largest population. It's actually the same question as the problem1 from leetcode above.

**> solution:**
```
select * from city;
```

```{sql, connection=con_mysql}
select 
    cn.Continent,
    cn.Name,
    sum(ci.Population) as population
from 
  city as ci
inner join country as cn on ci.CountryCode=cn.Code
group by
  ci.CountryCode
order by
  cn.Continent,
  population DESC;
```


```
select 
  tbl1.Continent,
  tbl1.Name,
  tbl1.population
from 
  (select 
      cn.Continent,
      cn.Name,
      sum(ci.Population) as population
  from 
    city as ci
  inner join country as cn on ci.CountryCode=cn.Code
  group by
    ci.CountryCode
  order by
    cn.Continent,
    population DESC) as tbl1
left join
   (select 
        cn.Continent,
        cn.Name,
        sum(ci.Population) as population
    from 
      city as ci
    inner join country as cn on ci.CountryCode=cn.Code
    group by
      ci.CountryCode
    order by
      cn.Continent,
      population DESC) as tbl2 
  on 
    tbl1.Continent=tbl2.Continent and
    tbl2.population > tbl1.population
where 
  tbl2.population is NULL;
```

**> note:**

* `inner join` should be before `group by`

> ### Find the largest country (by area) in each continent, show the continent, the name and the area:

**> idea:**
1. We will need a table with column `continent, name, area`.
2. We then want to get the largest country by area for each continent. 


**> solution:**
```
select * from country;
```

```
select 
  cn1.Continent, cn1.Name, cn1.SurfaceArea
from
  country as cn1
left join
  country as cn2 
on 
  cn1.Continent = cn2.Continent and -- for every continent in cn1, it will join with the same continent in cn2.
  cn2.SurfaceArea > cn1.SurfaceArea -- but we don't want to join country in each continent. We only want to join with the country which SurfaceArea is larger than the country in the left table. 
where 
  cn2.SurfaceArea is NULL; -- If the country in cn1 is the largest, then it will have no country from the right table to join. Therefore, it is the country we want. 
```

```
-- https://www.xaprb.com/blog/2006/12/07/how-to-select-the-firstleastmax-row-per-group-in-sql/
-- correlated subquery
SELECT -- not sure why -- ???
  Continent, Name, SurfaceArea
FROM country x
WHERE SurfaceArea >= ALL
    (SELECT SurfaceArea FROM country y
    WHERE y.continent=x.continent
    AND SurfaceArea>0);
```

In [ ]:
country.sort_values(by='SurfaceArea',ascending=False).groupby('Continent').head(1)[['Continent','Name','SurfaceArea']]

> ### List each continent and the name of the country that comes first alphabetically.

**> idea:**
1. we need to have a table have columns `continent, NAME`
2. keep only the country with its name comes first alphabetically

**> solution:**
```
select 
  cn1.Continent,
  cn1.Name 
from 
  country as cn1
left join
  country as cn2
on 
  cn1.Continent=cn2.Continent and
  cn2.Name < cn1.Name
where 
  cn2.Name is NULL;
```
```
SELECT Continent, Name
FROM country x
WHERE Name <= ALL(SELECT Name FROM country y WHERE y.Continent = x.Continent)
```

> ### Find the continents where all countries have a population <= 25000000. Then find the names of the countries associated with these continents. Show name, continent and population.

**> idea:**
1. we need a table with columns `Continent, country, population`

**> solution:**
```
-- ???
SELECT Name, Continent, population
FROM country x
WHERE 25000000  > ALL(SELECT population FROM country y WHERE x.Continent = y.Continent AND y.population > 0)

```

> ### Which countries have a GDP greater than every country in Europe? [Give the name only.] (Some countries may have NULL gdp values)

**> idea:**
1. select country name and then filter the rows using ALL command, continent!='Europe' and gdp!=NULL


**> solution:**
```
select Name
from country
where 
  GNP > ALL(select GNP from country where Continent="Europe" and GNP is NOT NULL) and 
  Continent!='Europe' and 
  GNP is NOT NULL;
```

---

> **Read csv**

In [ ]:
# no index column
df = pd.read_csv('data/final_set.csv')

# use the first column as index
df = pd.read_csv('data/final_set.csv', index_col=0)

> **Read Excel**

In [ ]:
df = pd.read_excel('data/WHR2018Chapter2OnlineData.xls', sheet_name='Table2.1')

> **Drop Columns**

In [ ]:
df.drop(['column1', 'column2'], axis=1, inplace=True)

In [ ]:
df = df.drop(['column1', 'column2'], axis=1)

> **Selecting columns in a pandas dataframe**

In [ ]:
df1 = df[['a','b']]

In [ ]:
df1 = df.iloc[:,0:2] # Remember that Python does not slice inclusive of the ending index.

In [ ]:
df1 = df.iloc[0,0:2].copy() # To avoid the case where changing df1 also changes df

[stackoverflow](https://stackoverflow.com/questions/11285613/selecting-columns-in-a-pandas-dataframe)

> **Check NA percentage for each column**

In [ ]:
df.isnull().sum()/df.shape[0]

> **Filter NA rows in a specific column**

In [ ]:
df = df[df.colname.notnull()]

> **Filter out rows with any NA**

In [ ]:
df[df.isnull().any(axis=1)]

> **Filter with multiple conditions**

In [ ]:
df[(df['country'].isin(final_set['country'])) & (df['year']==2008)].copy()
df[(df['country'].isin(final_set['country'])) | (df['year']==2008)].copy()
df[~df.countries.isin(countries)]

> **fill na in pandas**

In [ ]:
df.fillna('someNewValue', inplace=True)

In [ ]:
pivot

> **Save as csv**

In [ ]:
df.to_csv('name.csv', sep=',', index = False)

> **Converting a column within pandas dataframe from int to string**

In [ ]:
df['A']=df['A'].astype(int)

> **Create new column using some calculations on some current columns**

**More than 2 cagetory**

In [ ]:
df['newCol'] = df['oldCol'].apply(lambda x: x + 1)

In [ ]:
def getSeason(month):
    """
    match season from month
    """
    
    if (month in [12,1,2]):
        season = 'winter' 
    elif (month in [3,4,5]):
        season = 'spring' 
    elif (month in [6,7,8]):
        season = 'summer'
    else:
        season = 'fall'

    return season   

In [ ]:
tran_df['Transaction Month'].apply(lambda x: getSeason(x))

**Two category using list comprehension**

In [ ]:
df_freq['frequency'] = ['F=1' if t == 1 else 'F>1' for t in df_freq['TranCount']]

> **create confusion table**

In [ ]:
y_actu = pd.Series([2, 0, 2, 2, 0, 1, 1, 2, 2, 0, 1, 2], name='Actual’) 
y_pred = pd.Series([0, 0, 2, 1, 0, 2, 1, 0, 2, 0, 2, 2], name='Predicted’) 
df_confusion = pd.crosstab(y_actu, y_pred)

> **Replace column values**

In [ ]:
df['female'] = df['female'].map({'female': 1, 'male': 0})

In [ ]:
df.loc[df.female != 'female', 'female'] = 0
df.loc[df.female == 'female', 'female'] = 1

> **convert strings to date**

In [ ]:
pd.to_datetime('2018/02/26 00:00:00')

In [ ]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])

without date

In [ ]:
pd.to_datetime('2018/02/26 00:00:00').date()

In [ ]:
df['Transaction Date'] = pd.to_datetime(df['Transaction Date']).apply(lambda x: x.date()) 

> **epoch/unit timestamp to datetime**

In [ ]:
 password_df['time'] = pd.to_datetime(password_df['time'], unit='ms')

> **convert strings to time**

In [ ]:
pd.to_datetime('06:44:50', format='%H:%M:%S')

In [ ]:
pd.to_datetime('06:44:50', format='%H:%M:%S').time()

In [ ]:
pd.to_datetime(df['Transaction Time'], format='%H:%M:%S').apply(lambda x: x.time()) 

> **get hour/date from datetime column, assigning to a new column**

In [ ]:
df['timestamp'] = df['timestamp'].astype('datetime64[ns]')
df['date'] = df['timestamp'].apply(lambda x: x.date()) 
df['weekday'] = df['timestamp'].apply(lambda x: x.weekday()) 
df['month'] = df['timestamp'].apply(lambda x: x.month)
df['hour'] = df['timestamp'].apply(lambda x: x.hour)
df['minute'] = df['timestamp'].apply(lambda x: x.minute)

In [ ]:
df['weekday'] = df['timestamp'].apply(lambda x: x.date().strftime('%A'))  # will return "Wednesday"
df['weekday'] = df['timestamp'].apply(lambda x: x.date().strftime('%a'))  # will return "Wed"

* [doc datetime](https://docs.python.org/3.3/library/datetime.html)
* [stack](https://stackoverflow.com/questions/36341484/get-day-name-from-weekday-int/36341648)
* [time symbol wildcard](https://docs.python.org/2/library/time.html)

> **How to get the value from a timedelta?**

In [ ]:
age = datetime.timedelta(12045)

In [ ]:
print(age.days)
print(age.total_seconds())

[stackoverflow](https://stackoverflow.com/questions/27322362/how-to-get-the-value-from-a-timedelta)

> **add days to dates in dataframe; datetime add days**

In [ ]:
df["X_DATE"] = df["DATE"] + timedelta(days=180)

In [ ]:
# add month
df["X_DATE"] = df["DATE"] + pd.offsets.MonthOffset(1)

[stack](https://stackoverflow.com/questions/46741423/add-months-to-a-date-in-pandas)

> **rbind, cbind**

In [ ]:
# rbind
pd.concat([df1, df2])

# cbind
pd.concat([df1, df2], axis=1)

> **Pivot method**

In [ ]:
df = pd.DataFrame({'foo': ['one','one','one','two','two','two'],
                       'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                       'baz': [1, 2, 3, 4, 5, 6]})
df

In [ ]:
df.pivot(index='foo', columns='bar', values='baz')

> **pivot table method**

In [ ]:
# use it when we need to have more than one column as index
df_output = df.pivot_table(index=['col1', 'col2'], columns='metric', values='count')

> **Gather columns into rows by using melt**

[doc](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)

In [ ]:
df = pd.DataFrame({'country': ['Taiwan','Japan'],
                    '2011': [1, 2],
                    '2012': [3, 4],
                    '2013': [5, 6],
                    '2014': [7, 8],                  
                  })
df

In [ ]:
df.melt(id_vars='country')#,var_name='year', value_name='value'

> **Bin values based on ranges with pandas**

[stackoverflow](https://stackoverflow.com/questions/31736671/bin-values-based-on-ranges-with-pandas)

[doc](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.cut.html)

In [ ]:
pd.cut(x = pd.Series([1,5,75,125]), 
       bins = [0, 50, 100,200]
      labels = [1,2,3,4])

> **Print full dataframe in jupyter notebook**

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

> **Getting the Row which has the max value in groups using groupby**

In [ ]:
idx = df.groupby(['Profile ID'])['count'].transform(max) == df['count']
df[idx]

[stackoverflow](https://stackoverflow.com/questions/15705630/python-getting-the-row-which-has-the-max-value-in-groups-using-groupby)

> **normalize columns in pandas**

In [ ]:
# Split data into response and predictors
y = train_binary_dummy['isNDF']
x = train_binary_dummy.drop('isNDF', axis=1)

In [ ]:
from sklearn.preprocessing import scale
def my_scaler(x):
    """standardize the predictors"""
    
    new_x = pd.DataFrame(scale(x, axis=0, with_mean=True, with_std=True, copy=True))
    new_x.columns = x.columns
    
    return new_x

In [ ]:
x = my_scaler(x)

> **Drop duplicate rows in Python Pandas/ keep unique rows**

In [ ]:
import pandas as pd
df = pd.DataFrame({"A":["foo", "foo", "foo", "bar"], "B":[0,1,1,1], "C":["A","A","B","A"]})
df.drop_duplicates(subset=['A', 'C'], keep=False)

keep : {‘first’, ‘last’, False}, default ‘first’

[drop_duplicates doc](http://pandas.pydata.org/pandas-docs/version/0.17/generated/pandas.DataFrame.drop_duplicates.html)

[stackoverflow](https://stackoverflow.com/questions/23667369/drop-all-duplicate-rows-in-python-pandas)

> **Read and flatten json into pandas dataframe**

In [ ]:
import pandas as pd
import json
from pandas.io.json import json_normalize

with open('../data/mpd.slice.0-999.json') as f:
    data = json.load(f)

song_df = json_normalize(data=data['playlists'], 
                         record_path='tracks', 
                         meta=['collaborative', 'duration_ms', 'modified_at', 'name', 
                               'num_albums', 'num_artists', 'num_edits', 'num_followers', 'num_tracks','pid', 'description'],
                         record_prefix='track_',
                         errors='ignore')

# record_path: the nested key that we want to flatten
# meta: the keys in the first level
# record_prefix: the prefix for the keys in the nested level
# error = ‘ignore’ : will ignore KeyError if keys listed in meta are not always present

* [JSON to pandas DataFrame](https://stackoverflow.com/questions/21104592/json-to-pandas-dataframe)
* [Document: pandas.io.json.json_normalize](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html)
* [Quick Tutorial: Flatten Nested JSON in Pandas
](https://www.kaggle.com/jboysen/quick-tutorial-flatten-nested-json-in-pandas)

> **Get difference between two lists**

In [ ]:
temp1 = ['One', 'Two', 'Three', 'Four']
temp2 = ['One', 'Two']
list(set(temp1) - set(temp2))

> **group by count percentage for a categorical column**

In [6]:
df = pd.DataFrame({'foo': ['one','one','one','two','two','two'],
                       'bar': ['A', 'A', 'B', 'B', 'C', 'C'],
                       'value': [1, 2, 3, 4, 5, 6]})
df

,bar,foo,value
0,A,one,1
1,A,one,2
2,B,one,3
3,B,two,4
4,C,two,5
5,C,two,6


If we want to calculate the percentage of A using the sum of value for each group in foo, we can use `crosstab` from pandas

In [7]:
percentage_df = pd.crosstab(index=df['foo'],
                              columns=df['bar'],
                              values=df['value'],
                              aggfunc=np.sum,
                              dropna=False,
                              normalize='index') 
percentage_df = percentage_df.reset_index()
percentage_df.columns.name = None
percentage_df

,foo,A,B,C
0,one,0.5,0.500000,0.000000
1,two,0.0,0.266667,0.733333


[crosstab doc](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.crosstab.html)

> **pandas create new column based on values from other columns**

In [ ]:
def label_race (row):
    if row['eri_hispanic'] == 1:
        result = 'Hispanic'
    elif row['eri_afr_amer'] + row['eri_asian'] + row['eri_hawaiian'] + row['eri_nat_amer'] + row['eri_white'] > 1 :
        result = 'Two Or More'
    else:     
        result = 'White'
    return result

In [ ]:
df.apply (lambda row: label_race (row),axis=1)

[stackoverflow](https://stackoverflow.com/questions/26886653/pandas-create-new-column-based-on-values-from-other-columns)

> **Pandas: use groupby to count difference between dates**

[stackoverflow](https://stackoverflow.com/questions/38915186/pandas-use-groupby-to-count-difference-between-dates)

In [ ]:
days_id = pd.DataFrame(duplicated_activate_records.groupby('id_os')['datepartition'].apply(lambda x: x.iloc[0] - x.iloc[-1])).reset_index()


> **get culmulative sum of a column**

In [ ]:
df['cum_sum'] = df.val1.cumsum()
df['cum_perc'] = 100*df.cum_sum/df.val1.sum()

> **filter string contains**

[stack](https://stackoverflow.com/questions/27975069/how-to-filter-rows-containing-a-string-pattern-from-a-pandas-dataframe/27975230)

In [ ]:
df[df['ids'].str.contains("ball")]

> **create a sequence**

[stack](https://stackoverflow.com/questions/18265935/python-create-list-with-numbers-between-2-values)

In [ ]:
range(0, 7*12+1, 7)

> **faster way to transform string to datetime**

[reddit](https://www.reddit.com/r/learnpython/comments/6evlv5/faster_ways_to_convert_from_string_to_datetime/)

In [ ]:
date_cache = {k: pd.to_datetime(k) for k in df['date'].unique()}
df['date'] = df['date'].map(date_cache)

> **sum columns**

In [ ]:
df.sum(axis=1)

> **select columns using iloc**

In [ ]:
df.iloc[:,1:len(df.columns)] # select columns starting from the second column to the last

> **divide multiple columns by another column in pandas**

[stack](https://stackoverflow.com/questions/34540567/divide-multiple-columns-by-another-column-in-pandas)

In [ ]:
mydf= pd.DataFrame({'A':['a','b','c','d'],'D':[1,2,5,4],'E':[2,2,4,6],'F':[5,3,6,2]})
mydf

In [ ]:
mydf[['E','F']] = mydf[['E','F']].div(mydf.D, axis=0)
mydf

In [ ]:
# equivalent to this
mydf.iloc[:, 2:len(mydf.columns)] = mydf.iloc[:, 1:len(mydf.columns)].div(mydf.D, axis=0)

> **print whole dataframe**

In [ ]:
print(df.to_string())

> **opposite of crosstab/pivot**

In [ ]:
mydf= pd.DataFrame({'A':['a','b','c','d'],'D':[1,2,5,4],'E':[2,2,4,6],'F':[5,3,6,2]})
mydf

In [ ]:
pd.melt(mydf, id_vars=['A'])

> **multiple aggregations of the same column**

[stack](https://stackoverflow.com/questions/17578115/pass-percentiles-to-pandas-agg-function)

In [ ]:
df.groupby("dummy").agg({"value": [np.mean, np.sum]})


In [ ]:
# percentile
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [ ]:
df.groupby("dummy").agg({
    'value': [np.sum, np.mean, np.std, np.median, 
              np.min, percentile(25), percentile(75), np.max, 
              np.mean]})

> **sample a subset of rows**

[pandas doc](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html)
    

In [ ]:
df.sample(frac=0.1, replace=False) # default false
df.sample(frac=0.1, replace=True)
df.sample(random_state=1)

> **save model and load model sklearn**

[blog](https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)

In [ ]:
# Fit the model on 33%
model = LogisticRegression()
model.fit(X_train, Y_train)
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

> **see how long the python script runs**

In [ ]:
from datetime import datetime
startTime = datetime.now()
print startTime

# do a lot of things

endTime = datetime.now()
print startTime
print endTime
print startTime-endTime

> **convert categorical feature/column into numerical code**

[stack](https://stackoverflow.com/questions/32011359/convert-categorical-data-in-pandas-dataframe)

In [ ]:
# check dataframe type
df.dtypes

# set categorical features
categorical = ['a','b']

# transform multiple columns into categorical dtype
df[categorical] = df[categorical].astype('category')

# convert categorical feature into numerical code
df[categorical] = df[categorical].apply(lambda x: x.cat.codes)

In [ ]:
a = pd.DataFrame({'id': [1,2,2],
                    'device': ['device1', 'device2','device3']              
                  })

b = pd.DataFrame({'id': [1,1,2],
                    'device': ['device1', 'device5','device6']              
                  })

> **sort values by multiple columns**

In [ ]:
raw.sort_values(by=['Product Rank'], , ascending=[True, False])

> **custom aggregation function using multiple columns**

* [Learn More About Pandas By Building and Using a Weighted Average Function](http://pbpython.com/weighted-average.html)

In [ ]:
def wavg(group, avg_name, weight_name):
    """ http://stackoverflow.com/questions/10951341/pandas-dataframe-aggregate-function-using-multiple-columns
    In rare instance, we may not have weights, so just return the mean. Customize this if your business case
    should return otherwise.
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [ ]:
wavg(sales, "Current_Price", "Quantity")

In [ ]:
sales.groupby("Manager").apply(wavg, "Current_Price", "Quantity")

> **rank by group**

[link](http://www.datasciencemadesimple.com/rank-dataframe-python-pandas-min-max-dense-rank-group/)

In [ ]:
df_3["group_rank"] = df_3.groupby(firstDim)[metric].rank(ascending=False,method='min')

> **combine two column as list**

In [ ]:
big_table['combine']=big_table[['a', 'b','c']].values.tolist()

> **remove column with all zeros**

[stack](https://stackoverflow.com/questions/21164910/delete-column-in-pandas-if-it-is-all-zeros)

In [ ]:
df.loc[:, (df != 0).any(axis=0)]

> **create dataframe pandas**

In [4]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
df

,col1,col2
0,1,3
1,2,4


> **rename combine multiple level columns | flatten a hierarchical index in columns**

* [How to flatten a hierarchical index in columns](https://stackoverflow.com/questions/14507794/python-pandas-how-to-flatten-a-hierarchical-index-in-columns)

In [9]:
mydf= pd.DataFrame({'A':['a','a','c','c'],'B':[1,2,5,4]})
mydf

,A,B
0,a,1
1,a,2
2,c,5
3,c,4


In [12]:
finaldf = mydf.groupby('A').agg({'B':[np.mean, np.median]}).reset_index()
finaldf

A    B       
     mean median
0  a  1.5    1.5
1  c  4.5    4.5

In [11]:
finaldf.columns = ['_'.join(col).strip() if col[1]!='' else col[0] for col in finaldf.columns.values]
finaldf

,A,B_mean,B_median
0,a,1.5,1.5
1,c,4.5,4.5


In [ ]:
c=pd.merge(a,b,on='id',how='outer')
c['same_device'] = c['device_x'] == c['device_y']
c

In [ ]:
c.groupby('id')['same_device'].sum()

In [ ]:
from datetime import date

In [ ]:
d0 = date(2018, 1, 28)
d1 = date(2018, 7, 28)
delta = d1 - d0
print delta.days